In [8]:
# -*- coding: utf-8 -*-
"""

"""
# dit opnieuw runnen als email weer eens error geeft:
#pip install Aspose.Email-for-Python-via-NET
import os
import copy
import email as pymail
from email import policy
from email.parser import BytesParser
import nltk
from nltk.stem import WordNetLemmatizer
import time
from datetime import datetime
import re
import pandas as pd
import aspose
import lxml.html
from aspose import email
#from aspose.email.mapi import MapiMessage

class DataFactory:
    """
    extracting the data from the e-mail messages, cleaning the data and
    applying potential pre-processing methods
    """

    COPYRIGHT = ['***********',
                 'EDRM Enron Email Data Set has been produced in EML, PST and NSF format by ZL Technologies, Inc. This Data Set is licensed under a Creative Commons Attribution 3.0 United States License <http://creativecommons.org/licenses/by/3.0/us/> . To provide attribution, please cite to ZL Technologies, Inc. (http://www.zlti.com).',
                 '***********']

    data_frame = pd.DataFrame()

    email_data = []
    
    
    
    
    def __init__(self, project):
        """intialize the cleaning and pre-processing process"""
        self.start_time = time.time()
        self.last_time = time.time()
        self.proj = project

        if 'ENRON' in project:
            file_name = 'reload/data_frame_ENRON.zip'
        elif 'KPMG_W' in project:
            file_name = 'reload/data_frame_KPMG_W.zip'
        elif 'KPMG_2' in project:
            file_name = 'reload/data_frame_KPMG_2.zip'

        self.stopwords_en = self.get_stopwords()

        if os.path.isfile(file_name):
            # when there is already a pickled version, use it
            self.data_frame = pd.read_pickle(file_name, compression='zip')
            print('--> loaded dataframe file')
        else:
            # when there is not, find out where the data_dir is and create a
            # new dataframe with the e-mails
            print('--> started factory')
            self.columns = ['TIMESTAMP', 'FROM', 'TO', 'SUBJECT', 'CONTENT',
                            'ATTACHMENTS', 'IMPORTANCE', 'RESPONSIVE']
            mail_list = self.get_mails(start_dir=self.get_start_dir())
            print(len(mail_list))
            self.scholtes_prep()
            self.data_frame = self.create_dataframe(mail_list)
            self.data_frame['TOKENS_BODY_SUBJ'] = self.data_frame.apply(
                    lambda row: row['TOKENS'] + row['SUBJ_TOKENS'], axis=1)

            # drop the empty emails (those without timestamp)
            self.data_frame = self.data_frame[
                    self.data_frame.TIMESTAMP.map(lambda d: str(d)) != 'None']
            self.data_frame = self.data_frame.reset_index(drop=True)

            self.data_frame.to_pickle(path='C:\\Users\\tim_s\\Downloads\\Thesis KPMG Tim Smit\\ENRONpreprocessed.pkl', compression='zip')


            print('--> initation factory took {}s'
                  .format(int(time.time()-self.start_time)))

    def get_start_dir(self):
        """
        Based upon the current absolute path, find out whether we are on the
        """
        # Note to Tim: path to your data-directory
        start_dir = 'C:\\Users\\tim_s\\Downloads\\Used mail enron\\Used mailboxes RevisedEDRMv1_Complete'

        if 'ENRON' in self.proj:
            data_dir = start_dir + '\\ENRON data export'

        return data_dir

    def report_time(self, from_time):
        """
        record the time since the last time the time was recorded (or start)
        """
        new_time = time.time()
        diff = int(new_time - from_time)
        self.last_time = time.time()
        return str(diff) + 's'

    def get_filepaths(self, mail_dir):
        """
        return all paths for .msg files in the starting directory
        (including subfolders)
        """
        file_locs = []
        for subdir, dirs, files in os.walk(mail_dir):
            for file in files:
                split_filename = str(file).split('.')
                if split_filename[-1] == 'msg':
                    if dirs:
                        file_locs.append(file)
                    else:
                        file_name = subdir + '/' + file
                        file_name = file_name.replace('/', '\\')
                        file_locs.append(file_name)
                elif split_filename[-1] == 'eml':
                    if dirs:
                        file_locs.append(file)
                    else:
                        file_name = subdir + '/' + file
                        file_locs.append(file_name)

        return file_locs

    def get_mails(self, start_dir):
        """
        return a list of the documents based upon the directory they are in
        """

        if 'ENRON' in self.proj:
            senders = ['\\' + sub for sub in os.listdir(start_dir)]
            mails = []
            for sender in senders:
                rel_locs = self.get_filepaths(start_dir + sender + '\\Relevant')
                mails += [self.email_file(rel_loc, 1) for rel_loc in rel_locs]

                non_locs = self.get_filepaths(start_dir + sender +
                                              '\\Not Relevant')
                mails += [self.email_file(non_loc, 0) for non_loc in non_locs]

            self.email_data = mails

        print('--> loaded {} e-mails'.format(len(mails)))

        return mails

    def get_stopwords(self):
        """return the stopwords and extra added stopwords"""

        stop_words = nltk.corpus.stopwords.words('english') \
            + nltk.corpus.stopwords.words('dutch')

        stop_words += [word.replace("\'", '') for word in stop_words
                       if "\'" in word]
        for word in ['niet', 'no', 'not', 'nor', 'with', 'zonder', 'geen']:
            stop_words.remove(word)
        return stop_words

    def clean_body(self, input_txt):
        """clean the string and strip it of html"""
        string = str(input_txt)

        string = re.sub(r"\n", " ", string)
        string = re.sub(r"\'", " ", string)
        string = re.sub(r"\"", " ", string)

        for c_string in self.COPYRIGHT:
            if c_string in string:
                string = string.replace(c_string, " ")

        string = re.sub('\\s([0-9]+)\\s', ' NUMBERINORIGINAL ', string)
        string = re.sub("<!--?.*?-->", " ", string)
        string = re.sub(r"\\", " ", string)
        string = re.sub(r"\-", " ", string)
        string = re.sub(r"\t", " ", string)
        string = re.sub(r"\r", " ", string)
        string = re.sub(r"\f", " ", string)
        string = re.sub(r"\v", " ", string)
        string = re.sub("[^A-Za-z0-9(NUMBERINORIGINAL)\\s]", " ", string)
        string = re.sub(r"(\s\s)+", " ", string)

        return string.lower()

    def email_file(self, file_path, rel):
        """per msg file return the e-mail properties in a list"""
        def get_enron_date(datestring):
            return_str = ''
            parts = datestring.split('(')
            if parts:
                datestring = parts[0][:-1]
                return_str = datetime.strptime(datestring,
                                               '%a, %d %b %Y %H:%M:%S %z')
            return return_str

        def parse_to(to_type):
            'return the receivement type based upon recipient type'
            if to_type == 1:
                recipient = 'To'
            elif to_type == 2:
                recipient = 'Cc'
            elif to_type == 3:
                recipient = 'Bcc'
            return recipient

        if file_path.split('.')[-1] == 'msg':
            #print(vars(aspose))
            msg = aspose.email.mapi.MapiMessage.from_file(file_path)
            #aspose.email.mapi.
            e_date = get_enron_date(msg.headers.get('date'))
            #hier wordt een mail uitgelezen!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
            e_from = msg.sender_email_address
            e_to = []
            for rec in msg.recipients:
                e_to.append([rec.display_name.lower().replace('.', ''),
                             rec.email_address,
                             parse_to(rec.recipient_type)])
            e_subject = msg.subject
            e_content = self.clean_body(msg.body)
            e_atts = len(msg.attachments)
            # Note to Tim: haven't discovered the property importance in the Enron set yet,
            # included here since they are present in the KPMG set
            e_imp = 0

        return [e_date, e_from, e_to, e_subject, self.clean_body(e_content),
                e_atts, e_imp, rel]

    def create_dataframe(self, mail_list):
        """creates a dataframe based upon the filepaths method"""

        return pd.DataFrame(mail_list, columns=self.columns)

    def lemmatize_mails(self):
        """
        lemmatize each token and append the list of lemmas to the object
        """
        if 'PREP_TOKENS' not in self.columns:
            self.columns.append('TOKENS')

            lemmatizer = WordNetLemmatizer()
            stop_words = self.get_stopwords()

            for mail in self.email_data:
                content = mail[self.columns.index('CONTENT')]
                tokens = nltk.word_tokenize(content)

                mail_lemmas = []
                for token in tokens:
                    # do NOT take non-alphanumeric tokens and stop words into
                    # account
                    if token.isalnum() and token not in stop_words:
                        lemma = lemmatizer.lemmatize(token)
                        mail_lemmas.append(lemma)

                mail.append(mail_lemmas)
        print('--> lemmatized the e-mails')

    def scholtes_prep(self):
        """
        lemmatize each token and append the list of lemmas to the object
        """
        self.columns.append('TOKENS')
        self.columns.append('SUBJ_TOKENS')
        stop_words = self.get_stopwords()

        for mail in self.email_data:
            content = mail[self.columns.index('CONTENT')]
            tokens = nltk.word_tokenize(content)

            prep_tokens = []
            for token in tokens:
                # do NOT take non-alphanumeric tokens and stop words into
                # account
                if token.isalnum() and len(token) > 2 and \
                        token not in stop_words:
                    if token == 'numberinoriginal' or token.isnumeric():
                        token = '<NUM>'

                    prep_tokens.append(token)
            mail.append(prep_tokens)

            prep_tokens_subj = []

            subject = self.clean_body(mail[self.columns.index('SUBJECT')])
            subj_tokens = nltk.word_tokenize(subject)
            for token in subj_tokens:
                # do NOT take non-alphanumeric tokens and stop words into
                # account
                if token.isalnum() and len(token) > 2 and \
                        token not in stop_words:
                    if token == 'numberinoriginal' or token.isnumeric():
                        token = '<NUM>'

                    prep_tokens_subj.append(token)

            mail.append(prep_tokens_subj)
        print('--> pre-processed e-mails and subjects')


AttributeError: module 'aspose.email' has no attribute 'policy'

In [7]:
#pip install Aspose.Email-for-Python-via-NET
import aspose
from aspose import email

In [15]:
DataFactory('ENRON')

--> started factory
--> loaded 22044 e-mails
22044
--> pre-processed e-mails and subjects
--> initation factory took 159s


In [2]:
import pandas as pd

In [3]:
preprocessed=pd.read_pickle("C:\\Users\\tsmit\\Downloads\\KPMG thesis\\Data\\ENRONpreprocessed.pkl",compression='zip')
preprocessed['CONTENT+SUBJECT']=[' '.join(preprocessed.loc[index]['TOKENS_BODY_SUBJ']) for index in preprocessed.index]

In [57]:
preprocessed[preprocessed['RESPONSIVE']==1]

,TIMESTAMP,FROM,TO,SUBJECT,CONTENT,ATTACHMENTS,IMPORTANCE,RESPONSIVE,TOKENS,SUBJ_TOKENS,TOKENS_BODY_SUBJ,CONTENT+SUBJECT
0,2001-02-06 19:10:00-08:00,David W Delainey,"[[steven j kean, Steven J Kean, To]]",2001 EES Goals,steve write these down in pencil i need a chan...,1,0,1,"[steve, write, pencil, need, chance, review, t...","[<NUM>, ees, goals]","[steve, write, pencil, need, chance, review, t...",steve write pencil need chance review thanks d...
1,2000-11-10 12:48:00-08:00,David W Delainey,"[[tim belden, Tim Belden, To], [john j lavorat...",2001 Plan,tim lets not worry about margin at this point ...,0,0,1,"[tim, lets, not, worry, margin, point, trying,...","[<NUM>, plan]","[tim, lets, not, worry, margin, point, trying,...",tim lets not worry margin point trying get han...
2,2001-01-02 02:15:00-08:00,David W Delainey,"[[wes colwell, Wes Colwell, To], [faith killen...",2001 Pro Forma,guys attached you will find a completed number...,1,0,1,"[guys, attached, find, completed, <NUM>, pro, ...","[<NUM>, pro, forma]","[guys, attached, find, completed, <NUM>, pro, ...",guys attached find completed <NUM> pro forma b...
3,2001-01-19 20:17:00-08:00,David W Delainey,"[[mark e haedicke, Mark E Haedicke, To]]",California Action and Next Steps Meeting. Mond...,mark you should be there if possible regards d...,0,0,1,"[mark, possible, regards, delainey, forwarded,...","[california, action, next, steps, meeting, mon...","[mark, possible, regards, delainey, forwarded,...",mark possible regards delainey forwarded david...
4,2001-01-19 09:34:00-08:00,David W Delainey,"[[william s bradford, William S Bradford, To],...",California Exposure Table,can we realistically net transmission costs ag...,1,0,1,"[realistically, net, transmission, costs, ctc,...","[california, exposure, table]","[realistically, net, transmission, costs, ctc,...",realistically net transmission costs ctc under...
5,2000-12-12 12:31:00-08:00,David W Delainey,"[[wes colwell, Wes Colwell, To], [joseph deffn...",Catalytica,guys i suggest that we write it down in order ...,0,0,1,"[guys, suggest, write, order, ensure, get, bac...",[catalytica],"[guys, suggest, write, order, ensure, get, bac...",guys suggest write order ensure get back group...
6,2001-02-09 10:43:00-08:00,David W Delainey,"[[tim belden, Tim Belden, To]]",Congratulations,tim i appreciate the note and the comments i m...,0,0,1,"[tim, appreciate, note, comments, may, need, h...",[congratulations],"[tim, appreciate, note, comments, may, need, h...",tim appreciate note comments may need help fut...
7,2000-07-14 00:21:00-07:00,David W Delainey,"[[wes colwell, Wes Colwell, To]]",QBR,wes we have the following dates set up a) east...,0,0,1,"[wes, following, dates, set, east, midstream, ...",[qbr],"[wes, following, dates, set, east, midstream, ...",wes following dates set east midstream restruc...
8,2000-09-15 17:08:00-07:00,David W Delainey,"[[jeff donahue, Jeff Donahue, To]]",,forwarded by david w delainey hou ect on numb...,1,0,1,"[forwarded, david, delainey, hou, ect, <NUM>, ...",[],"[forwarded, david, delainey, hou, ect, <NUM>, ...",forwarded david delainey hou ect <NUM> <NUM> <...
3314,2001-10-24 09:56:22-07:00,Bobbie_Power@ENRON.net,"[[bowen jr raymond, Raymond.Bowen@ENRON.com, ...",Focus Group Survey Results,to management committee members from cindy ols...,1,0,1,"[management, committee, members, cindy, olson,...","[focus, group, survey, results]","[management, committee, members, cindy, olson,...",management committee members cindy olson beth ...


In [31]:
fraud=preprocessed[preprocessed['RESPONSIVE']==1]
not_fraud=preprocessed[preprocessed['RESPONSIVE']==0]

In [34]:
from sklearn.utils import resample
not_fraud_downsampled = resample(not_fraud,
                                replace = False, # sample without replacement
                                n_samples = len(fraud)*100, # match minority n
                                random_state = 27) # reproducible results
preprocessed = pd.concat([not_fraud_downsampled, fraud])

In [35]:
#preprocessed.loc[0]['TOKENS_BODY_SUBJ']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(preprocessed['TOKENS_BODY_SUBJ'], 
                                                    preprocessed['RESPONSIVE'], 
                                                    test_size=0.15, 
                                                    random_state=8)
#y_train.value_counts()
#y_test.value_counts()
X_train=X_train.astype(str)
X_test=X_test.astype(str)

In [36]:
#We'll use TF-IDF Vectors as features.
from sklearn.feature_extraction.text import TfidfVectorizer
ngram_range = (1,2)
min_df = 10
max_df = 1.
max_features = 300
tfidf = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        max_features=max_features,
                        norm='l2',
                        sublinear_tf=True)
                        
features_train = tfidf.fit_transform(X_train).toarray()
labels_train = y_train
print(features_train.shape)

features_test = tfidf.transform(X_test).toarray()
labels_test = y_test
print(features_test.shape)

(1888, 300)
(334, 300)


In [37]:
y_test.value_counts()

0    332
1      2
Name: RESPONSIVE, dtype: int64

In [38]:
# C
C = [.0001, .001, .01]

# gamma
gamma = [.0001, .001, .01, .1, 1, 10, 100]

# degree
degree = [1, 2, 3, 4, 5]

# kernel
kernel = ['linear', 'rbf', 'poly']

# probability
probability = [True]

# Create the random grid
random_grid = {'C': C,
              'kernel': kernel,
              'gamma': gamma,
              'degree': degree,
              'probability': probability
             }

print(random_grid)

{'C': [0.0001, 0.001, 0.01], 'kernel': ['linear', 'rbf', 'poly'], 'gamma': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100], 'degree': [1, 2, 3, 4, 5], 'probability': [True]}


In [39]:
# First create the base model to tune
from sklearn.model_selection import RandomizedSearchCV
svc = svm.SVC(random_state=8)

# Definition of the random search
random_search = RandomizedSearchCV(estimator=svc,
                                   param_distributions=random_grid,
                                   n_iter=50,
                                   scoring='f1',
                                   cv=3, 
                                   verbose=1, 
                                   random_state=8)

# Fit the random search model
random_search.fit(features_train, labels_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\tsmit\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\tsmit\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\tsmit\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\tsmit\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being se

C:\Users\tsmit\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\tsmit\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\tsmit\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\tsmit\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_f

C:\Users\tsmit\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\tsmit\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\tsmit\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\tsmit\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_f

C:\Users\tsmit\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\tsmit\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\tsmit\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\tsmit\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_f

C:\Users\tsmit\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\tsmit\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\tsmit\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\tsmit\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_f

RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=SVC(C=1.0, cache_size=200, class_weight=None,
                                 coef0=0.0, decision_function_shape='ovr',
                                 degree=3, gamma='auto_deprecated',
                                 kernel='rbf', max_iter=-1, probability=False,
                                 random_state=8, shrinking=True, tol=0.001,
                                 verbose=False),
                   iid='warn', n_iter=50, n_jobs=None,
                   param_distributions={'C': [0.0001, 0.001, 0.01],
                                        'degree': [1, 2, 3, 4, 5],
                                        'gamma': [0.0001, 0.001, 0.01, 0.1, 1,
                                                  10, 100],
                                        'kernel': ['linear', 'rbf', 'poly'],
                                        'probability': [True]},
                   pre_dispatch='2*n_jobs', random

In [42]:
random_search.confusion_matrix()

AttributeError: 'RandomizedSearchCV' object has no attribute 'confusion_matrix'

In [40]:
print("The best hyperparameters from Random Search are:")
print(random_search.best_params_)
print("")
print("The mean accuracy of a model with these hyperparameters is:")
print(random_search.best_score_)

The best hyperparameters from Random Search are:
{'probability': True, 'kernel': 'poly', 'gamma': 0.001, 'degree': 4, 'C': 0.01}

The mean accuracy of a model with these hyperparameters is:
0.0


In [71]:
labels_train.value_counts()

0    18716
1       21
Name: RESPONSIVE, dtype: int64

In [73]:
https://github.com/miguelfzafra/Latest-News-Classifier/blob/master/0.%20Latest%20News%20Classifier/04.%20Model%20Training/07.%20MT%20-%20SVM.ipynb

0.9988792229278967

In [5]:
import os
os.chdir('Z:\\Scriptie JPebesma\\Project_Whiskey\\Relevant')
os.getcwd()

'Z:\\Scriptie JPebesma\\Project_Whiskey\\Relevant'

In [15]:
os.chdir('Z:\\Scriptie JPebesma\\Code\\reload')
#project2020=pd.read_pickle('data_frame_KPMG_2.zip')
projectWhiskey=pd.read_pickle('data_frame_KPMG_W.zip')
os.chdir('Z:\\Scriptie TSmit\\Pickles')
#project2020.to_pickle(path='Z:\\Scriptie TSmit\\Pickles\\2020.pkl',compression='zip')
projectWhiskey.to_pickle(path='Z:\\Scriptie TSmit\\Pickles\\Whiskey.pkl',compression='zip')

In [10]:
project2020.columns

Index(['TIMESTAMP', 'FROM', 'TO', 'SUBJECT', 'CONTENT', 'ATTACHMENTS',
       'IMPORTANCE', 'RESPONSIVE', 'TOKENS', 'SUBJ_TOKENS',
       'TOKENS_BODY_SUBJ'],
      dtype='object')